In [ ]:
# if you have NOT installed the webscraping package yet, now install the webscraping package
!conda install -c conda-forge beautifulsoup4 --yes
# install the required parsor:
!conda install -c conda-forge lxml --yes
!conda install -c conda-forge html5lib --yes
!conda install -c conda-forge requests --yes

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

# Part 1
## Get HTML data from wikipedia

In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

In [2]:
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(URL)
soup = BeautifulSoup(page.content, 'lxml')

In [3]:
# Use Pandas to read the Wikipedia source page and convert its table to a dataframe
table = pd.read_html(URL)
table[0].columns = table[0].iloc[0]
df_table = table[0]
df_table

,M1A,Not assigned,nan
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


In [4]:
# Dataframe will consist of 3 columns: PostalCode, Borough, and Neighborhood
df_table.columns = ['PostalCode', 'Borough', 'Neighborhood']

# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df_table = df_table[df_table.Borough != 'Not assigned']

# If a cell has a borough but a Not assigned neighborhood, set neighborhood name to its borough.
df_table.Neighborhood.replace('Not assigned',df_table.Borough,inplace=True)

# Combine neighborhoods with same postal code in a single row and separate with commas (as_index=False to retain columns)
df_table = df_table.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x:','.join(x))
df_table

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
9,M1N,Scarborough,Birch Cliff / Cliffside West


In [5]:
# Summary of rows in processed table
df_table.shape

(103, 3)

# Part 2
## Achieve Latitude and Longitude of each PostalCode

In [9]:
! pip install geocoder
import geocoder # import geocoder
"""
# initialize your variable to None
lat_lng_coords = None
postal_code = df_table['PostalCode']

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]"""

     |████████████████████████████████| 102kB 16.7MB/s ta 0:00:01


"\n# initialize your variable to None\nlat_lng_coords = None\npostal_code = df_table['PostalCode']\n\n# loop until you get the coordinates\nwhile(lat_lng_coords is None):\n  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))\n  lat_lng_coords = g.latlng\n\nlatitude = lat_lng_coords[0]\nlongitude = lat_lng_coords[1]"

In [6]:
df_geospace = pd.read_csv('http://cocl.us/Geospatial_data')
df_geospace.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

df_final = pd.merge(df_table, df_geospace, on = 'PostalCode')

In [7]:
df_final

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476
9,M1N,Scarborough,Birch Cliff / Cliffside West,43.692657,-79.264848


# Part 3       Analysis of the City of Toronto
## 1. Create a map of Toronto with neighborhoods superimposed on top.

In [ ]:
# IMPORT NECESSARY LIBRARIES

import geocoder # import geocoder
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [14]:
!pip install folium
# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 92kB 17.1MB/s eta 0:00:01
Libraries imported.


In [15]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [16]:
import folium

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Borough'], df_final['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Versions

In [17]:
CLIENT_ID = '4WCHLC132BAJVEFOFEE3KAZN1JU1CJO0A0LLA2PE3FKTXBX1' # your Foursquare ID
CLIENT_SECRET = 'PL5U4WH2YLDYFD2EJNVCVME0FINHUGKQ00HN5IUCV0JJJUBL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4WCHLC132BAJVEFOFEE3KAZN1JU1CJO0A0LLA2PE3FKTXBX1
CLIENT_SECRET:PL5U4WH2YLDYFD2EJNVCVME0FINHUGKQ00HN5IUCV0JJJUBL


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [18]:
toronto_data=df_final

#check if 'Richmond Hill' belongs to any Neighborhood
toronto_data[toronto_data['Neighborhood'].astype(str).str.contains('Richmond')]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
58,M5H,Downtown Toronto,Richmond / Adelaide / King,43.650571,-79.384568


In [ ]:
toronto_data.loc[58, 'Neighborhood']

Get the neighborhood's latitude and longitude values.

In [19]:
neighborhood_latitude = toronto_data.loc[58, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[58, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[58, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Richmond / Adelaide / King are 43.65057120000001, -79.3845675.


#### Now, let's get the top 100 venues that are in Richmond within a radius of 500 meters.

In [20]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=4WCHLC132BAJVEFOFEE3KAZN1JU1CJO0A0LLA2PE3FKTXBX1&client_secret=PL5U4WH2YLDYFD2EJNVCVME0FINHUGKQ00HN5IUCV0JJJUBL&v=20180605&ll=43.65057120000001,-79.3845675&radius=500&limit=100'

In [21]:
results = requests.get(url).json()
# results

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Four Seasons Centre for the Performing Arts,Concert Hall,43.650592,-79.385806
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Rosalinda,Vegetarian / Vegan Restaurant,43.650252,-79.385156
3,The Keg Steakhouse + Bar - York Street,Restaurant,43.649987,-79.384103
4,Cafe Landwer,Café,43.648753,-79.385367


And how many venues were returned by Foursquare?

In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

94 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    print("Finished!")
    
    return(nearby_venues)

### Now write the code to run the above function on each neighborhood and create a new dataframe called toronto_venues.

In [28]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'],
                                    radius = 3000
                                  )


Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview / Henry Farm / Oriole
Bayview Village
York Mills / Silver Hills
Willowdale / Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Bathurst Manor / Wilson Heights / Downsview North
Northwood Park / York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill / Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West / Riverdale
India Bazaar / The Beaches 

#### Let's check the size of the resulting dataframe

In [29]:
print(toronto_venues.shape)
toronto_venues.head()

(9808, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malvern / Rouge,43.806686,-79.194353,Toronto Pan Am Sports Centre,43.790623,-79.193869,Athletics & Sports
1,Malvern / Rouge,43.806686,-79.194353,African Rainforest Pavilion,43.817725,-79.183433,Zoo Exhibit
2,Malvern / Rouge,43.806686,-79.194353,Polar Bear Exhibit,43.823372,-79.185145,Zoo
3,Malvern / Rouge,43.806686,-79.194353,Toronto Zoo,43.820582,-79.181551,Zoo
4,Malvern / Rouge,43.806686,-79.194353,Orangutan Exhibit,43.818413,-79.182548,Zoo Exhibit


#### Let's check how many venues were returned for each neighborhood

In [197]:
toronto_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,100,100,100,100,100,100
Alderwood / Long Branch,100,100,100,100,100,100
Bathurst Manor / Wilson Heights / Downsview North,100,100,100,100,100,100
Bayview Village,100,100,100,100,100,100
Bedford Park / Lawrence Manor East,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [30]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 296 uniques categories.


## 3. Analyze Each Neighborhood

In [31]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo Exhibit,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train 

and let's exam the new dataframe size

In [32]:
toronto_onehot.shape

(9808, 296)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [33]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Zoo Exhibit,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Leather Goods Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pide Place,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train 

#### Let's confirm the new size

In [34]:
toronto_grouped.shape

(98, 296)

#### Let's print each neighborhood along with the top 5 most common venues

In [35]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                venue  freq
0  Chinese Restaurant  0.07
1   Indian Restaurant  0.05
2          Restaurant  0.05
3            Pharmacy  0.04
4      Clothing Store  0.04


----Alderwood / Long Branch----
                       venue  freq
0                Coffee Shop  0.05
1               Burger Joint  0.04
2              Grocery Store  0.04
3           Department Store  0.03
4  Middle Eastern Restaurant  0.03


----Bathurst Manor / Wilson Heights / Downsview North----
                       venue  freq
0                Coffee Shop  0.12
1              Grocery Store  0.05
2              Deli / Bodega  0.03
3  Middle Eastern Restaurant  0.03
4             Sandwich Place  0.03


----Bayview Village----
               venue  freq
0        Coffee Shop  0.10
1        Pizza Place  0.06
2  Korean Restaurant  0.06
3               Bank  0.05
4     Sandwich Place  0.05


----Bedford Park / Lawrence Manor East----
              venue  freq
0       Coffee Shop  0.09
1    Clothing S

#### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Indian Restaurant,Restaurant,Coffee Shop,Clothing Store,Fast Food Restaurant,Caribbean Restaurant,Pharmacy,Bookstore,Bubble Tea Shop
1,Alderwood / Long Branch,Coffee Shop,Grocery Store,Burger Joint,Restaurant,Middle Eastern Restaurant,Burrito Place,Breakfast Spot,Furniture / Home Store,Seafood Restaurant,Department Store
2,Bathurst Manor / Wilson Heights / Downsview North,Coffee Shop,Grocery Store,Japanese Restaurant,Deli / Bodega,Sandwich Place,Café,Gym / Fitness Center,Middle Eastern Restaurant,Sushi Restaurant,Burger Joint
3,Bayview Village,Coffee Shop,Pizza Place,Korean Restaurant,Bank,Chinese Restaurant,Sandwich Place,Furniture / Home Store,Gas Station,Pharmacy,Bubble Tea Shop
4,Bedford Park / Lawrence Manor East,Coffee Shop,Clothing Store,Bakery,Sushi Restaurant,Café,Park,Pub,Tea Room,Bank,Bagel Shop


## 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [38]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([3, 3, 3, 2, 3, 1, 2, 4, 4, 4, 4, 3, 3, 1, 4, 1, 3, 1, 1, 4, 4, 4,
       3, 3, 2, 4, 4, 2, 3, 1, 4, 1, 3, 3, 2, 1, 4, 3, 2, 2, 4, 4, 2, 2,
       1, 2, 3, 4, 4, 4, 2, 3, 3, 4, 4, 2, 4, 2, 2, 4, 4, 2, 3, 1, 1, 1,
       4, 4, 2, 4, 4, 2, 2, 1, 1, 3, 1, 4, 4, 4, 4, 4, 4, 3, 1, 1, 0, 3,
       2, 2, 2, 3, 3, 3, 2, 4, 3, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [39]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# since data is long, there are some missing values, replace them with 0, and then change Cluster Labels to integers
toronto_merged = toronto_merged.fillna(0)
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype(int)

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,2,Zoo Exhibit,Fast Food Restaurant,Pizza Place,Coffee Shop,Park,Trail,Gas Station,Restaurant,Bank,Grocery Store
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,2,Coffee Shop,Park,Pizza Place,Fast Food Restaurant,Gas Station,Bank,Pharmacy,Sandwich Place,Grocery Store,Burger Joint
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,2,Coffee Shop,Fast Food Restaurant,Pizza Place,Sandwich Place,Grocery Store,Park,Burger Joint,Hotel,Beer Store,Bank
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Bank,Sandwich Place,Fast Food Restaurant,Pharmacy,Pizza Place,Gas Station,Indian Restaurant,Discount Store,Beer Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3,Coffee Shop,Chinese Restaurant,Clothing Store,Indian Restaurant,Pharmacy,Gym,Restaurant,Bank,Fast Food Restaurant,Bakery


Finally, let's visualize the resulting clusters

In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

#### Cluster 1

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Scarborough,0,Zoo Exhibit,Park,Zoo,Golf Course,Grocery Store,Farm,Café,Dessert Shop,Gift Shop,Electronics Store


#### Cluster 2

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Scarborough,1,Coffee Shop,Park,Ice Cream Shop,Hotel,Beach,Sandwich Place,Café,Restaurant,Sushi Restaurant,Bistro
51,Downtown Toronto,1,Coffee Shop,Park,Historic Site,Restaurant,Café,Farmers Market,Italian Restaurant,Dance Studio,Bakery,Pub
52,Downtown Toronto,1,Coffee Shop,Park,Restaurant,Farmers Market,Spa,Hotel,Café,Vegetarian / Vegan Restaurant,Historic Site,Italian Restaurant
53,Downtown Toronto,1,Coffee Shop,Park,Café,Farmers Market,Bakery,Restaurant,Plaza,Hotel,Dance Studio,Mediterranean Restaurant
54,Downtown Toronto,1,Coffee Shop,Park,Hotel,Restaurant,Farmers Market,Café,Diner,Vegetarian / Vegan Restaurant,Bakery,Supermarket
55,Downtown Toronto,1,Coffee Shop,Park,Café,Restaurant,Vegetarian / Vegan Restaurant,Farmers Market,Yoga Studio,Plaza,Theater,Sandwich Place
56,Downtown Toronto,1,Coffee Shop,Park,Café,Plaza,Restaurant,Sandwich Place,Farmers Market,Yoga Studio,Supermarket,Liquor Store
57,Downtown Toronto,1,Coffee Shop,Diner,Café,Japanese Restaurant,Hotel,Theater,Restaurant,Vegetarian / Vegan Restaurant,Park,Gastropub
58,Downtown Toronto,1,Coffee Shop,Café,Park,Farmers Market,Restaurant,Hotel,Bakery,Japanese Restaurant,Sandwich Place,Gym
59,Downtown Toronto,1,Coffee Shop,Café,Yoga Studio,Plaza,Gym,Park,Restaurant,Japanese Restaurant,Italian Restaurant,Farmers Market


#### Cluster 3

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,2,Zoo Exhibit,Fast Food Restaurant,Pizza Place,Coffee Shop,Park,Trail,Gas Station,Restaurant,Bank,Grocery Store
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,2,Coffee Shop,Park,Pizza Place,Fast Food Restaurant,Gas Station,Bank,Pharmacy,Sandwich Place,Grocery Store,Burger Joint
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,2,Coffee Shop,Fast Food Restaurant,Pizza Place,Sandwich Place,Grocery Store,Park,Burger Joint,Hotel,Beer Store,Bank
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2,Coffee Shop,Bank,Sandwich Place,Fast Food Restaurant,Pharmacy,Pizza Place,Gas Station,Indian Restaurant,Discount Store,Beer Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3,Coffee Shop,Chinese Restaurant,Clothing Store,Indian Restaurant,Pharmacy,Gym,Restaurant,Bank,Fast Food Restaurant,Bakery
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2,Coffee Shop,Pharmacy,Fast Food Restaurant,Sandwich Place,Pizza Place,Bank,Grocery Store,Park,Ice Cream Shop,Chinese Restaurant
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029,2,Coffee Shop,Sandwich Place,Chinese Restaurant,Bank,Burger Joint,Department Store,Ice Cream Shop,Gas Station,Pizza Place,Pharmacy
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577,3,Coffee Shop,Grocery Store,Bank,Middle Eastern Restaurant,Sandwich Place,Chinese Restaurant,Gym / Fitness Center,Department Store,Park,Ice Cream Shop
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476,1,Coffee Shop,Park,Ice Cream Shop,Hotel,Beach,Sandwich Place,Café,Restaurant,Sushi Restaurant,Bistro
9,M1N,Scarborough,Birch Cliff / Cliffside West,43.692657,-79.264848,2,Coffee Shop,Park,Ice Cream Shop,Pub,Pizza Place,Sandwich Place,Grocery Store,Bank,Burger Joint,Mexican Restaurant


#### Cluster 4

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,3,Coffee Shop,Chinese Restaurant,Clothing Store,Indian Restaurant,Pharmacy,Gym,Restaurant,Bank,Fast Food Restaurant,Bakery
7,Scarborough,3,Coffee Shop,Grocery Store,Bank,Middle Eastern Restaurant,Sandwich Place,Chinese Restaurant,Gym / Fitness Center,Department Store,Park,Ice Cream Shop
10,Scarborough,3,Coffee Shop,Pharmacy,Chinese Restaurant,Supermarket,Pizza Place,Burger Joint,Sandwich Place,Fast Food Restaurant,Bookstore,Middle Eastern Restaurant
11,Scarborough,3,Chinese Restaurant,Middle Eastern Restaurant,Coffee Shop,Grocery Store,Supermarket,Burger Joint,Burrito Place,Restaurant,Pizza Place,Pet Store
12,Scarborough,3,Chinese Restaurant,Indian Restaurant,Restaurant,Coffee Shop,Clothing Store,Fast Food Restaurant,Caribbean Restaurant,Pharmacy,Bookstore,Bubble Tea Shop
13,Scarborough,3,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Middle Eastern Restaurant,Noodle House,Restaurant,Caribbean Restaurant,Vietnamese Restaurant,Burrito Place,Pharmacy
14,Scarborough,3,Chinese Restaurant,Bubble Tea Shop,Coffee Shop,Pharmacy,Bakery,Vietnamese Restaurant,Gas Station,Japanese Restaurant,Pizza Place,Asian Restaurant
15,Scarborough,3,Chinese Restaurant,Bakery,Bubble Tea Shop,Japanese Restaurant,Coffee Shop,Vietnamese Restaurant,Dessert Shop,Supermarket,Pharmacy,Noodle House
17,North York,3,Chinese Restaurant,Bank,Pharmacy,Japanese Restaurant,Gas Station,Bakery,Coffee Shop,Pizza Place,Sandwich Place,Caribbean Restaurant
18,North York,3,Coffee Shop,Chinese Restaurant,Bakery,Pharmacy,Middle Eastern Restaurant,Restaurant,Japanese Restaurant,Supermarket,Fast Food Restaurant,Seafood Restaurant


#### Cluster 5

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,East York,4,Café,Coffee Shop,Indian Restaurant,Bakery,Gastropub,Park,Brewery,Greek Restaurant,BBQ Joint,Pakistani Restaurant
37,East Toronto,4,Park,Coffee Shop,Beach,Pub,Café,Breakfast Spot,Bakery,Ice Cream Shop,BBQ Joint,Indian Restaurant
38,East York,4,Italian Restaurant,Park,Grocery Store,Coffee Shop,Bakery,Supermarket,Café,Restaurant,Trail,Bookstore
40,East York,4,Café,Park,Ice Cream Shop,Greek Restaurant,Bakery,Brewery,Coffee Shop,Gastropub,Cocktail Bar,American Restaurant
41,East Toronto,4,Park,Bakery,Café,Coffee Shop,Brewery,Greek Restaurant,Gastropub,Diner,Ice Cream Shop,American Restaurant
42,East Toronto,4,Park,Beach,Coffee Shop,Café,Brewery,Bakery,Gastropub,BBQ Joint,American Restaurant,Ice Cream Shop
43,East Toronto,4,Park,Coffee Shop,Brewery,Greek Restaurant,Bakery,Pizza Place,Diner,Beach,Ice Cream Shop,BBQ Joint
44,Central Toronto,4,Bakery,Italian Restaurant,Café,Coffee Shop,Park,Sushi Restaurant,Restaurant,Burger Joint,Indian Restaurant,Ice Cream Shop
45,Central Toronto,4,Bakery,Coffee Shop,Italian Restaurant,Café,Park,Indian Restaurant,Tea Room,Sporting Goods Shop,Bagel Shop,Restaurant
46,Central Toronto,4,Bakery,Coffee Shop,Café,Italian Restaurant,Park,Bagel Shop,Sushi Restaurant,Indian Restaurant,Tea Room,Thai Restaurant
